## Import

In [1]:
import random
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [2]:
device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')

In [3]:
print (f"PyTorch version:{torch.__version__}") # 1.12.1 이상
print(f"MPS 장치를 지원하도록 build 되었는지: {torch.backends.mps.is_built()}") # True 여야 합니다.
print(f"MPS 장치가 사용 가능한지: {torch.backends.mps.is_available()}") # True 여야 합니다.
!python -c 'import platform;print(platform.platform())'

PyTorch version:2.0.0
MPS 장치를 지원하도록 build 되었는지: True
MPS 장치가 사용 가능한지: True
macOS-13.4.1-arm64-arm-64bit


## Hyperparameter Setting

In [4]:
# setting parameter
CFG = {
    'TRAIN_WINDOW_SIZE':90, # 90일치로 학습
    'PREDICT_SIZE':21, # 21일치 예측
    'EPOCHS':20,
    'LEARNING_RATE':0.0001, # 0.001 -> 0.01 -> 0.0001
    'BATCH_SIZE':64, # 128 -> 256 -> 512 -> 128
    'SEED':41
}

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed(난수) 고정

### 데이터 불러오기

In [6]:
train_data = pd.read_csv('open/train.csv').drop(columns=['ID', '제품'])

In [7]:
# Load sales.csv
sales_data = pd.read_csv('open/sales.csv')
brand_data = pd.read_csv('open/brand_keyword_cnt.csv')
product_data = pd.read_csv('open/product_info.csv')

In [8]:
train_data.shape, sales_data.shape, brand_data.shape,product_data.shape 

((15890, 463), (15890, 465), (3170, 460), (12778, 2))

### 데이터 전처리

In [9]:
# 숫자형 변수들의 min-max scaling을 수행하
numeric_cols = train_data.columns[4:]
# 칵 column의 min 및 max 계산
min_values = train_data[numeric_cols].min(axis=1)
max_values = train_data[numeric_cols].max(axis=1)
# 각 행의 범위(max-min)를 계산하고, 범위가 0인 경우 1로 대체
ranges = max_values - min_values
ranges[ranges == 0] = 1
# min-max scaling 수행
train_data[numeric_cols] = (train_data[numeric_cols].subtract(min_values, axis=0)).div(ranges, axis=0)
# max와 min 값을 dictionary 형태로 저장
scale_min_dict = min_values.to_dict()
scale_max_dict = max_values.to_dict()

In [10]:
# # Label Encoding
# label_encoder = LabelEncoder()
# categorical_columns = ['대분류', '중분류', '소분류', '브랜드']

# for col in categorical_columns:
#     label_encoder.fit(train_data[col])
#     train_data[col] = label_encoder.transform(train_data[col])

In [15]:
def make_train_data(data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):
    STEP_SIZE = 10
    
    num_rows = len(data)
    window_size = train_size + predict_size
    adjusted_size = (len(data.columns) - window_size + 1) // STEP_SIZE
    
    label_encoders = {}  # LabelEncoder 객체를 저장할 딕셔너리
    
    # 판매량과 판매금액을 제외한 열 목록
    feature_columns = ['대분류', '중분류', '소분류', '브랜드']
    
    for col in feature_columns:
        le = LabelEncoder()
        le.fit(data[col])  # 각 카테고리 데이터에 Label Encoding 수행
        data[col] = le.transform(data[col])  # 원본 데이터를 변환한 값으로 갱신
        label_encoders[col] = le  # 딕셔너리에 LabelEncoder 객체 저장
    
    input_data = np.empty((num_rows * adjusted_size, train_size, len(feature_columns) + 1))  # 추가된 특징에 대한 차원 추가
    target_data = np.empty((num_rows * adjusted_size, predict_size))

    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :len(feature_columns)])  # 판매량 및 판매금액을 제외한 정보
        sales_data = np.array(data.iloc[i, len(feature_columns):])

        for j in range(0, len(sales_data) - window_size + 1, STEP_SIZE):
            window = sales_data[j: j + window_size]
            sales_feature = get_sales_feature(data.index[j + train_size: j + window_size])  # 2022-01-01부터 2023-04-04까지의 판매금액 데이터
            temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), np.tile(window[:train_size], (1, len(feature_columns))), sales_feature))
            input_data[i * adjusted_size + j // STEP_SIZE] = temp_data
            target_data[i * adjusted_size + j // STEP_SIZE] = window[train_size:]

    return input_data, target_data
# 판매금액 데이터를 가져오는 함수 예시
def get_sales_feature(date_range):
    # 여기서 date_range에 해당하는 일별 판매금액을 가져오는 작업을 수행
    # 가져온 데이터를 Numpy 배열로 반환
    return np.array(sales_feature_data)

In [16]:
def make_predict_data(data, train_size=CFG['TRAIN_WINDOW_SIZE']):
    num_rows = len(data)
    input_data = np.empty((num_rows, train_size, len(data.iloc[0, len(feature_columns):])))  # 판매량 및 판매금액을 제외한 shape로 변경
    
    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, len(feature_columns):])  # 판매량 및 판매금액 정보 선택
        sales_data = np.array(data.iloc[i, -train_size:])
        
        window = sales_data[-train_size:]
        temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
        input_data[i] = temp_data
    
    return input_data

In [17]:
train_input, train_target = make_train_data(train_data)
test_input = make_predict_data(train_data)

  0%|          | 0/15890 [00:00<?, ?it/s]

NameError: name 'sales_feature_data' is not defined

In [ ]:
# Train / Validation Split
data_len = len(train_input)
val_input = train_input[-int(data_len*0.2):]
val_target = train_target[-int(data_len*0.2):]
train_input = train_input[:-int(data_len*0.2)]
train_target = train_target[:-int(data_len*0.2)]

In [ ]:
train_input.shape, train_target.shape, val_input.shape, val_target.shape, test_input.shape

### Custom Dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        
    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])
    
    def __len__(self):
        return len(self.X)

In [ ]:
train_dataset = CustomDataset(train_input, train_target)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_input, val_target)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

### 모델 선언

In [ ]:
class LSTMModel(nn.Module):
    # 모델 복잡성 증가 256 -> 512
    def __init__(self, input_size=5, hidden_size=512, output_size=CFG['PREDICT_SIZE']):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size//2),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hidden_size//2, output_size)
        )
            
        self.dropout = nn.Dropout(0.5)  # 드롭아웃 레이어 추가
        self.actv = nn.ReLU()
    
    def forward(self, x):
        # x shape: (B, TRAIN_WINDOW_SIZE, 5)
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size, x.device)
        
        # LSTM layer
        lstm_out, hidden = self.lstm(x, hidden)
        
        # Only use the last output sequence
        last_output = self.dropout(lstm_out[:, -1, :])  # Dropout 적용
        
        # Fully connected layer
        output = self.actv(self.fc(last_output))
        
        return output.squeeze(1)
    
    def init_hidden(self, batch_size, device):
        # Initialize hidden state and cell state
        return (torch.zeros(1, batch_size, self.hidden_size, device=device),
                torch.zeros(1, batch_size, self.hidden_size, device=device))

### 모델 학습

In [ ]:
# Clear GPU memory after each epoch
def train(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    best_loss = float('inf')
    best_model = None

    for epoch in range(1, CFG['EPOCHS'] + 1):
        model.train()
        train_loss = []

        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)

            optimizer.zero_grad()

            output = model(X)
            loss = criterion(output, Y)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        val_loss = validation(model, val_loader, criterion, device)
        print(f'Epoch: [{epoch}] Train Loss: [{np.mean(train_loss):.5f}] Val Loss: [{val_loss:.5f}]')

        if best_loss > val_loss:
            best_loss = val_loss
            best_model = model
            print('Model Saved')
            
        # Clear GPU memory
        torch.cuda.empty_cache()

    return best_model

In [ ]:
def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []
    
    with torch.no_grad():
        for X, Y in tqdm(iter(val_loader)):
            X = X.to(device)
            Y = Y.to(device)
            
            output = model(X)
            loss = criterion(output, Y)
            
            val_loss.append(loss.item())
    return np.mean(val_loss)

## Run !!

In [ ]:
model = LSTMModel()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
infer_model = train(model, optimizer, train_loader, val_loader, device)

## 모델 추론

In [ ]:
test_dataset = CustomDataset(test_input, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    predictions = []
    
    with torch.no_grad():
        for X in tqdm(iter(test_loader)):
            X = X.to(device)
            
            output = model(X)
            
            # 모델 출력인 output을 CPU로 이동하고 numpy 배열로 변환
            output = output.cpu().numpy()
            
            predictions.extend(output)
    
    return np.array(predictions)

In [ ]:
pred = inference(infer_model, test_loader, device)

In [ ]:
# 추론 결과를 inverse scaling
for idx in range(len(pred)):
    pred[idx, :] = pred[idx, :] * (scale_max_dict[idx] - scale_min_dict[idx]) + scale_min_dict[idx]
    
# 결과 후처리
pred = np.round(pred, 0).astype(int)

In [ ]:
pred.shape

## Submission

In [ ]:
submit = pd.read_csv('open/sample_submission.csv')
submit.head()

In [ ]:
submit.iloc[:,1:] = pred
submit.head()

In [ ]:
submit.to_csv('open/baseline_submit.csv', index=False)